In [1]:
import random
import itertools as it
import time

import pandas as pd
import numpy as np
np.random.seed(1337)


dataset_A = pd.read_csv('../TSPA.csv', sep=';', names=['x', 'y', 'cost'])
dataset_B = pd.read_csv('../TSPB.csv', sep=';', names=['x', 'y', 'cost'])

datasets = [dataset_A, dataset_B]

dataset_A.shape, dataset_B.shape

((200, 3), (200, 3))

In [2]:
node1, node2 = dataset_B.iloc[177], dataset_B.iloc[43]

print(f'{node1}\n\n{node2}')

x       1450
y        398
cost      52
Name: 177, dtype: int64

x       1654
y       1486
cost     638
Name: 43, dtype: int64


In [3]:
def euclidean_distance(node1, node2):
    return np.int32(((node1['x'] - node2['x']) ** 2 + (node1['y'] - node2['y']) ** 2) ** 0.5 + 0.5)

In [4]:
print(f'distance between node1 and node2 = {euclidean_distance(node1, node2)}')

distance between node1 and node2 = 1107


In [5]:
def nodes_cost(node1, node2):
    return node1['cost'] + node2['cost']

In [6]:
print(f'objective function of node1 and node2 = {euclidean_distance(node1, node2) + nodes_cost(node1, node2)}')

objective function of node1 and node2 = 1797


In [7]:
def calculate_function_cost(dataset):
    cost = dataset["cost"].sum()
    for i in range(-1, len(dataset) - 1):
        cost += euclidean_distance(dataset.iloc[i], dataset.iloc[i + 1])
    return int(cost)

In [8]:
def calculate_distance_matrix(dataset):
    n_nodes = len(dataset)
    distance_matrix = np.zeros((n_nodes, n_nodes), dtype=np.float64)
    for i, j in it.product(range(n_nodes), range(n_nodes)):
        if i == j:
            continue
        a, b = dataset.iloc[i], dataset.iloc[j]
        distance_matrix[i, j] = euclidean_distance(a, b) + nodes_cost(a, b)
    return distance_matrix

In [9]:
distance_matrix_A = calculate_distance_matrix(dataset_A)
distance_matrix_B = calculate_distance_matrix(dataset_B)

In [10]:
print(f'objective function of node1 and node2 = {distance_matrix_A[177, 43]}')

objective function of node1 and node2 = 1831.0


In [11]:
import matplotlib.pyplot as plt

import seaborn as sns

def plot(dataset, solution, title):
    max_x = dataset['x'].max()
    max_y = dataset['y'].max()

    aspect_ratio = int(max_x / max_y)

    if max_x > max_y:
        height = 6.0
        width = height * aspect_ratio
    else:
        width = 6.0
        height = width * aspect_ratio

    sns.set_theme(rc={'figure.figsize': (width, height)})

    sns.scatterplot(
        data=dataset,
        x='x',
        y='y',
        size='cost',
        legend=False
    )

    for i in range(-1, len(solution) - 1):
        idx1 = solution[i]
        idx2 = solution[i + 1]
        x_values = [dataset.loc[idx1, "x"], dataset.loc[idx2, "x"]]
        y_values = [dataset.loc[idx1, "y"], dataset.loc[idx2, "y"]]
        plt.plot(
            x_values,
            y_values,
            color="b",
            linestyle="-",
        )
    
    plt.title(title)
    plt.show();

In [12]:
def precompute_candidate_edges(distance_matrix, node_costs):
    candidate_edges = {}
    for node in range(len(distance_matrix)):
        nearest = np.argsort(distance_matrix[node])[:5]
        candidate_edges[node] = list(nearest)
    return candidate_edges

# Local Search



In [13]:
def init_random_solution(dataset: pd.DataFrame, distance_matrix: pd.DataFrame, start: int) -> pd.DataFrame:
    return dataset.sample(n=int(len(dataset) * 0.5 + 0.5))

In [ ]:
def objective_change_two_nodes(dm: np.ndarray, solution: list, i: int, j: int) -> float:
    if i == j:
        return 0.0

    n = len(solution)
    a, b = solution[i], solution[j]

    a_prev = solution[i - 1] if i > 0 else solution[-1]
    a_next = solution[(i + 1) % n]
    b_prev = solution[j - 1] if j > 0 else solution[-1]
    b_next = solution[(j + 1) % n]

    # Edges to be removed and added
    edges_removed = []
    edges_added = []

    # Remove edges connected to a and b
    if a_prev not in (a, b):
        edges_removed.append((a_prev, a))
        edges_added.append((a_prev, b))
    if a_next not in (a, b):
        edges_removed.append((a, a_next))
        edges_added.append((b, a_next))
    if b_prev not in (a, b):
        edges_removed.append((b_prev, b))
        edges_added.append((b_prev, a))
    if b_next not in (a, b):
        edges_removed.append((b, b_next))
        edges_added.append((a, b_next))

    # Calculate delta
    delta = -sum(dm[u, v] for u, v in edges_removed) + sum(
        dm[u, v] for u, v in edges_added
    )

    return delta


def objective_change_two_edges(dm: np.ndarray, solution: list, i: int, j: int) -> float:
    if i >= j or (i == 0 and j == len(solution) - 1):
        return 0.0

    n = len(solution)
    a_prev = solution[i - 1] if i > 0 else solution[-1]
    a = solution[i]
    b = solution[j]
    b_next = solution[(j + 1) % n]

    # Edges before and after reversal
    cost_before = dm[a_prev, a] + dm[b, b_next]
    cost_after = dm[a_prev, b] + dm[a, b_next]

    delta = cost_after - cost_before

    return delta


def objective_change_inter_route(
    dm: np.ndarray, solution: list, i: int, vacant_node: int, node_costs: list
) -> float:
    n = len(solution)
    node_in_solution = solution[i]
    prev_node = solution[i - 1] if i > 0 else solution[-1]
    next_node = solution[(i + 1) % n]

    # Edge costs before and after the swap
    edge_cost_before = dm[prev_node, node_in_solution] + dm[node_in_solution, next_node]
    edge_cost_after = dm[prev_node, vacant_node] + dm[vacant_node, next_node]

    # Node costs before and after the swap
    node_cost_before = node_costs[node_in_solution]
    node_cost_after = node_costs[vacant_node]

    delta = (edge_cost_after - node_cost_after) - (edge_cost_before - node_cost_before)

    return delta


def two_nodes_exchange(solution: list, i: int, j: int) -> list:
    new_solution = solution.copy()
    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution


def two_edges_exchange(solution: list, i: int, j: int) -> list:
    if i >= j:
        return solution.copy()  # No change if i >= j

    new_solution = solution.copy()
    new_solution[i : j + 1] = new_solution[i : j + 1][::-1]
    return new_solution


def inter_route_swap(
    solution: list,
    i: int,
    vacant_node: int,
    selected_nodes: set,
    non_selected_nodes: set,
) -> tuple:
    new_solution = solution.copy()
    node_in_solution = new_solution[i]
    new_solution[i] = vacant_node

    # Update the node sets
    selected_nodes = selected_nodes.copy()
    non_selected_nodes = non_selected_nodes.copy()

    selected_nodes.remove(node_in_solution)
    selected_nodes.add(vacant_node)
    non_selected_nodes.remove(vacant_node)
    non_selected_nodes.add(node_in_solution)

    return new_solution, selected_nodes, non_selected_nodes


def get_remaining_nodes(selected_nodes: set, num_nodes: int) -> set:
    return set(range(num_nodes)) - selected_nodes


def browse_intra_solutions(dm, solution, candidate_edges):
    intra_neighbors = []
    n = len(solution)
    for i in range(n):
        for j in range(i+1, n):
            # Check if the new edges introduce at least one candidate edge
            next_i = (i + 1) % n
            prev_j = (j - 1) % n
            if (solution[j] in candidate_edges[solution[next_i]] or
                solution[i] in candidate_edges[solution[prev_j]]):
                delta = objective_change_two_edges(dm, solution, i, j)
                if delta < 0:
                    intra_neighbors.append((i, j, delta, "edge"))
    return intra_neighbors


def browse_inter_solutions(dm, solution, non_selected_nodes, candidate_edges, node_costs):
    inter_neighbors = []
    for i in range(len(solution)):
        for vacant_node in non_selected_nodes:
            # Check if introducing at least one candidate edge
            if (vacant_node in candidate_edges[solution[i]] or
                solution[i] in candidate_edges[vacant_node]):
                delta = objective_change_inter_route(dm, solution, i, vacant_node, node_costs)
                if delta < 0:
                    inter_neighbors.append((i, vacant_node, delta, "inter"))
    return inter_neighbors


def update_solution(solution, best_neighbor, selected_nodes, non_selected_nodes):
    neighbor_type = best_neighbor[-1]
    if neighbor_type == "edge":
        i, j = best_neighbor[:2]
        # Perform two-edge exchange
        solution = two_edges_exchange(solution, i, j)
    elif neighbor_type == "inter":
        i, vacant_node = best_neighbor[:2]
        # Swap selected and non-selected nodes
        solution[i] = vacant_node
        selected_nodes.add(vacant_node)
        selected_nodes.remove(solution[i])
        non_selected_nodes.add(solution[i])
        non_selected_nodes.remove(vacant_node)
    return solution, selected_nodes, non_selected_nodes


def local_search(dataset: pd.DataFrame, distance_matrix: np.ndarray, initial_solution: list, candidate_edges: dict, strategy = "greedy", intra_search = "edge", debug_mode = True) -> pd.DataFrame:
    num_nodes = len(distance_matrix)
    initial_cost = calculate_function_cost(dataset.loc[initial_solution])

    solution = initial_solution.copy()
    selected_nodes = set(initial_solution)
    non_selected_nodes = get_remaining_nodes(selected_nodes, num_nodes)

    while True:
        intra_neighbors = browse_intra_solutions(distance_matrix, solution, candidate_edges)
        # Filter intra neighbors to include only those introducing candidate edges
        filtered_intra = []
        for neighbor in intra_neighbors:
            i, j, delta, move_type = neighbor
            # Identify new edges introduced by the move
            new_edge1 = (solution[i], solution[j-1])
            new_edge2 = (solution[j], solution[i+1])
            # Check if at least one new edge is a candidate edge
            if (new_edge1[1] in candidate_edges[new_edge1[0]] or
                new_edge2[1] in candidate_edges[new_edge2[0]]):
                filtered_intra.append(neighbor)
        
        # Generate inter-route neighbors
        inter_neighbors = browse_inter_solutions(distance_matrix, solution, non_selected_nodes, candidate_edges)
        # Filter inter neighbors to include only those introducing candidate edges
        filtered_inter = []
        for neighbor in inter_neighbors:
            i, vacant_node, delta, move_type = neighbor
            # Identify new edges introduced by the move
            new_edge1 = (solution[i-1], vacant_node)
            new_edge2 = (vacant_node, solution[i+1])
            # Check if at least one new edge is a candidate edge
            if (vacant_node in candidate_edges[solution[i-1]] or
                solution[i+1] in candidate_edges[vacant_node]):
                filtered_inter.append(neighbor)
        
        # Combine filtered neighbors
        all_neighbors = filtered_intra + filtered_inter

        if not all_neighbors:
            # No improvement found
            break

        if strategy == "greedy":
            random.shuffle(all_neighbors)
            best_neighbor = next((n for n in all_neighbors if n[2] < 0), None)
        elif strategy == "steepest":
            candidates = [n for n in all_neighbors if n[2] < 0]
            if not candidates:
                break
            best_neighbor = min(candidates, key=lambda x: x[2])

        if best_neighbor is None:
            # No improving neighbor found
            break

        # Save the old solution for debugging
        old_solution = solution.copy()

        # Update solution and cost
        solution, selected_nodes, non_selected_nodes = update_solution(
            solution, best_neighbor, selected_nodes, non_selected_nodes
        )
        initial_cost += best_neighbor[2]

        if debug_mode:
            # Calculate real improvement
            real_improvement = calculate_function_cost(dataset.loc[old_solution]) - calculate_function_cost(
                dataset.loc[solution]
            )

            if real_improvement != -best_neighbor[2]:
                print(f"Promised improvement: {best_neighbor[2]}")
                print(f"Real improvement: {real_improvement}")
                print(f"Operation: {best_neighbor[-1]}")
                print("===========")

            assert calculate_function_cost(dataset.loc[solution]) < calculate_function_cost(dataset.loc[old_solution])

    return dataset.loc[solution]

In [15]:
def init_local_search(
    dataset: pd.DataFrame,
    distance_matrix: np.ndarray,
    start: int,
    candidate_edges: np.ndarray,
) -> pd.DataFrame:
    solution = list(init_random_solution(dataset, distance_matrix, start).index)
    
    solution = local_search(dataset, distance_matrix, solution, candidate_edges, strategy='random', intra_search='edge', debug_mode=False)
    
    return solution

In [16]:
def experiment(dataset, distance_matrix, candidate_edges):
    start_time = time.time()
    ratings = []
    for i in range(200):
        local_search_solution = init_local_search(dataset, distance_matrix, start=i, candidate_edges=candidate_edges)
        solution = list(local_search_solution.index)
        ratings.append((solution, calculate_function_cost(local_search_solution)))

    best = sorted(ratings, key=lambda x: x[1])[0]
    minimum = sorted(ratings, key=lambda x: x[1])[0][1]
    mean = sum([obj_function for _, obj_function in ratings]) / len(ratings)
    maximum = sorted(ratings, key=lambda x: x[1])[-1][1]
    end_time = time.time()

    print(f"Best solution: {best[0]}")
    print(f"Time to calculate: {(end_time - start_time):.4f}")
    print("Objective function statistics:")
    print(f"{minimum = }\n{mean = }\n{maximum = }")
    plot(dataset, best[0], title='Candidate moves in local search')

In [17]:
node_costs = dataset_A['cost'].values
candidate_edges = precompute_candidate_edges(distance_matrix_A, node_costs)
experiment(dataset_A, distance_matrix_A, candidate_edges)

IndexError: string index out of range

In [ ]:
node_costs = dataset_B['cost'].values
candidate_edges = precompute_candidate_edges(distance_matrix_B, node_costs)
experiment(dataset_B, distance_matrix_B, candidate_edges, node_costs, 'candidates')